<a href="https://colab.research.google.com/github/marriamaslova/compling_nlp_hse_course/blob/master/notebooks/lm_intro/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [ ]:
! pip install razde

In [ ]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
from collections import Counter
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
from scipy.sparse import lil_matrix

In [ ]:
dvach = open('2ch_corpus.txt').read()
news = open('lenta.txt').read()

In [ ]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [ ]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [ ]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)

In [ ]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [ ]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [ ]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [ ]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, 3))

unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, 3))

In [ ]:
import itertools

In [ ]:
matrix_dvach = lil_matrix((len(bigrams_dvach), 
                   len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}
id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {bigram:i for i, bigram in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    trigram = word1, word2, word3
    bigram = word1, word2
    bigram_d = ' '.join(bigram)
    trigram_d = ' '.join(trigram)
    matrix_dvach[bigram2id_dvach[bigram_d], word2id_dvach[word3]] = (trigrams_dvach[trigram_d]/bigrams_dvach[bigram_d]) 

In [ ]:
matrix_news = lil_matrix((len(bigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}
id2bigram_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    trigram = word1, word2, word3
    bigram = word1, word2
    bigram_n = ' '.join(bigram)
    trigram_n = ' '.join(trigram)
    matrix_news[bigram2id_news[bigram_n], word2id_news[word3]] =  (trigrams_news[trigram_n]/
                                                                     bigrams_news[bigram_n])

In [ ]:
def generate(matrix, id2word, word2id, bigram2id, n=100, start='<start>, <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = bigram2id['<start>, <start>']
        current_idx = chosen
    
    return ' '.join(text)

In [ ]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [ ]:
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens)

In [ ]:
phrase_1 = 'Ранее появилась информация, что Twitter заблокировал аккаунты некоторых журналистов, в их числе корреспонденты телеканала CNN и газеты The Washington Post. Эти санкции были связаны с решением Twitter о запрете передавать текущее местоположение другого человека без его согласия. Лишившиеся доступа к аккаунтам журналисты писали о новой политике Маска, которая, по его словам, была вызвана инцидентом с преследованием его семьи во вторник вечером в Лос-Анджелесе. Бизнесмен позже провел опрос среди пользователей социальной сети по поводу разблокировки журналистов. Большая часть опрошенных (43%) высказались за немедленное разблокирование аккаунтов журналистов. Однако уже после публикации данных опроса Маск дезавуировал его результаты и заявил, что проведет опроc заново.'

In [ ]:
perplexity(*compute_joint_proba(phrase_1, probas_dvach))
#13539.339030311741

In [ ]:
perplexity(*compute_joint_proba(phrase_1, probas_news))
#5553.426834760291

In [ ]:
phrase_2 = 'Долбимся в ФУТочко. Притворяемся, что сделали 11 побед и подарили 9 оставшихся игр, хвалимся наградами, пулами, тимами, обсуждаем тактики. Но самое главное - кидаемся какашками друг в друга, по тому что кто-то играет не так, как нам хочется. Только пару дней им играю, обычно он на подыгрыше у меня оказывается , тот же смолярек лучше голы клепал , это наверное мои стиль игры такой , я второй волной забиваю, редко когда убегаю.Здесь мы жёстко бомбим на решение руководства клуба уволить Томаса Тухеля из-за нежелания покупать сорокалетнюю Кристину и поминаем его короткую, но великую эру в клубе - эру, когда Челси побывал в финалах почти всех соревнований, в которых он участвовал, и эру, в которой число 3 стало настоящим талисманом для клуба.'

In [ ]:
perplexity(*compute_joint_proba(phrase_2, probas_dvach))
#5750.225393577827

In [ ]:
perplexity(*compute_joint_proba(phrase_2, probas_news))
#5369.64628948354

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

2. Что такое сглаживание (smoothing)?